# Genus 2 case


So for a curve of genus $g=2$ we get
```{math}
f(x,y) = -y^2 + \lambda_0 x^5 + \lambda_2 x^4 + \lambda_4 x^3+ \lambda_6 x^2+ \lambda_8 x+ \lambda_{10},
```  
Let's load the appropriate SageMath library and define the curve $\mathscr{C}$

In [1]:
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface

In [2]:
# Defines the lambda coefficients
lambda0 = 1.0
lambda2 = 12.0
lambda4 = 2.0
lambda6  = 0.9
lambda8  = 21.0
lambda10  = 10.0

```{important}
The coefficients above must be real-floating-point numbers 
```

Since the current Sage library only works well on the field of rational numbers, we have to approximate all function coefficients by these numbers.


In [3]:
# Rational approximation
l0 = lambda0.nearby_rational(max_error=1e-10)
l2 = lambda2.nearby_rational(max_error=1e-10)
l4 = lambda4.nearby_rational(max_error=1e-10)
l6 = lambda6.nearby_rational(max_error=1e-10)
l8 = lambda8.nearby_rational(max_error=1e-10)
l10 = lambda10.nearby_rational(max_error=1e-10)

Next, we need to define the variables and in the ring of polynomials over the rational numbers.

In [4]:
R.<x, y> = PolynomialRing(QQ, 2)

```{note}
The order of variables can be important depending on how you write your polynomial. Make sure you use the same order in your polynomial expression.
```

In [5]:
# Defining the polynomial f
f = -y^2 + l0*x^5 + l2*x^4 + l4*x^3 + l6*x^2 + l8*x + l10

## Riemann surface

````{prf:definition}
:label: riemann_sur

A Riemann surface $X$ is a connected two-dimensional topological manifold with a complex-analytic structure on it. 
````

The SageMath library has a special function that allows us to generate the appropriate Riemann surface on which we will continue our work.

In [6]:
S = RiemannSurface(f, prec=100)

## Branch points

We can easily define a function that based on the $\lambda_{2i}$ coefficients will print all the branch points $e_i$ given by
```{math}
y^2 =(x-e_1)(x-e_2)(x-e_3)(x-e_4)(x-e_5).
```

In [7]:
def find_branch_points(ll0, ll2, ll4, ll6, ll8, ll10):
    # We define a ring of polynomials over the field of complex numbers
    CC_poly.<x> = PolynomialRing(CC)
    
    # We create a polynomial
    pol = ll0*x^5 + ll2*x^4 + ll4*x^3 + ll6*x^2 + ll8*x + ll10
    
    # We find roots
    roots = pol.roots(multiplicities=False)
    
    # We add a point at infinity if the degree of the polynomial is odd
    if pol.degree() % 2 == 1:
        roots.append(infinity)
    
    return roots

```{note}
The `multiplicities=False` parameter in the `roots()`  method in Sage has the following meaning:
- When `multiplicities=False`  (default): The method returns only the roots of the polynomial, without information about their multiplicities. The result is a list of unique root values.
- When `multiplicities=True`: The method returns pairs (root, multiplicity) for each root. The result is a list of tuples, where each tuple contains the root and its multiplicity.
```

In [8]:
# Example of use:

branch_points = find_branch_points(l0, l2, l4, l6, l8, l10)
print("Branch points:", branch_points)

Branch points: [-11.8251161409669, -1.05264146874943, -0.512317960119317, 0.695037784917798 - 1.04164546422316*I, 0.695037784917798 + 1.04164546422316*I, +Infinity]


## First and Second Kind Periods

In order to construct periodic functions we first have to define a canonical basis of the space of holomorphic differentials $\{du_i\mid i=1,\ldots,g\} $ and of associated meromorphic differentials $\{dr_i\mid i=1,\ldots,g\} $ on the Riemann surface by
```{math}
du_{2i-1} := \frac{x^{g-i} dx}{\partial_y f(x)},
```
```{math}
dr_{2i-1} := \frac{\mathcal{R}_{2i-1}(x) dx}{\partial_y f(x)}, 
```
where
```{math}
\mathcal{R}_{2i-1}(x)=\sum_{k=1}^{2i-1}k\lambda_{4i-2k-2}x^{g-i+k}.
```
We can write this in vector form for $g=2$ as follows 
```{math}
du= \begin{pmatrix} 
        x\\
        1
    \end{pmatrix} \frac{dx}{-2\sqrt{f(x)}},
```
```{math}
dr= \begin{pmatrix} 
        \mathcal{R}_{1}(x)\\
        \mathcal{R}_{3}(x)
    \end{pmatrix} \frac{dx}{-2\sqrt{f(x)}} = 
        \begin{pmatrix} 
            x^2\\
            3\lambda_0x^3 + 2\lambda_2 x^2 + \lambda_4 x
        \end{pmatrix} \frac{dx}{-2\sqrt{f(x)}},
```

The above holomorphic basis can be compared to what the `cohomology_basis()` function returns. In SageMath, `S.cohomology_basis()` returns this list of differentials, typically represented as polynomials  $g(x)$  corresponding to differentials:
```{math}
\omega = \frac{g(x) \, dx}{\partial f / \partial y}
```
where  $f(x, y) = 0$  defines the curve.


In [9]:
S.cohomology_basis()

[1, x]

It is visible that the order of elements is different from that adopted in {cite}`bernatska_computation_2024` convention. We will use Julia's one. 


###  First Kind Periods    

In [10]:
# holomorphic differentials base
holbais=[x,x^0]

Now, to calculate the first kind period matrices, we need to calculate the following integrals along the canonical homology cycles $\{ \mathfrak{a}_i, \mathfrak{b}_i\}_{i=1}^g$
```{math}
\omega = (\omega_{ij})= \left( \int_{\mathfrak{a}_j}du_i \right), \quad \omega' = (\omega'_{ij})= \left( \int_{\mathfrak{b}_j}du_i \right). 
```

 We can use the Sage function  `matrix_of_integral_values(differentials, integration_method='heuristic')` to compute the path integrals of the given differentials along the homology basis. The returned answer has a row for each differential. If the Riemann surface is given by the equation $y^2$, then the differentials are encoded by polynomials $g$, signifying the differential 
```{math}
g(x,y)\frac{dx}{(df/dy)}.
```
`Input:`
- `differentials` – a list of polynomials.
- `integration_method` – (default: 'heuristic'). String specifying the integration method to use. The options are 'heuristic' and 'rigorous'.

`Output:`  
A matrix, one row per differential, containing the values of the path integrals along the homology basis of the Riemann surface.

In [11]:
MofInt1=S.matrix_of_integral_values(holbais)
# Let's display the matrix in a shortened form so that it will be easy to see its structure
print(MofInt1.n(digits=5))

[     0.13760 - 0.66033*I  -4.9304e-32 - 0.91722*I  -3.9443e-31 + 0.40344*I       1.1746 + 0.25689*I]
[    -0.62448 + 0.28220*I -5.5467e-32 + 0.038862*I  -4.9304e-32 - 0.52554*I      0.14607 + 0.24334*I]


The structure of the returned matrix
```{math}
MofInt1 = 
    \begin{pmatrix}
        \omega_{1,1} & \omega_{1,2} & \omega'_{1,1} & \omega'_{1,2} \\
        \omega_{2,1} & \omega_{2,2} & \omega'_{2,1} & \omega'_{2,2}
    \end{pmatrix}
    =
    \begin{array}{|c|c|c|c|c|}
        \hline
        & \mathfrak{a}_1 & \mathfrak{a}_2 & \mathfrak{b}_1 & \mathfrak{b}_2 \\
        \hline
        x & \omega_{1,1} = \int_{\mathfrak{a}_1} du_1 & \omega_{1,2} = \int_{\mathfrak{a}_2} du_1 & \omega'_{1,1} = \int_{\mathfrak{b}_1} du_1 & \omega'_{1,2} = \int_{\mathfrak{b}_2} du_1 \\
        \hline
        1 & \omega_{2,1} = \int_{\mathfrak{a}_1} du_2 & \omega_{2,2} = \int_{\mathfrak{a}_2} du_2 & \omega'_{2,1} = \int_{\mathfrak{b}_1} du_2 & \omega'_{2,2} = \int_{\mathfrak{b}_2} du_2 \\
        \hline
    \end{array}
```

We can compare this with the results of the built-in Sage function: 'period_matrix()', which, for the adopted notational convention, will return a period matrix in the form
```{math}
    pM= \begin{pmatrix}
            \omega_{2,1} & \omega_{2,2} & \omega'_{2,1} & \omega'_{2,2} \\
            \omega_{1,1} & \omega_{1,2} & \omega'_{1,1} & \omega'_{2,1}
    \end{pmatrix}  
```

In [12]:
pM=S.period_matrix()
print(pM.n(digits=5))

[   -0.62448 + 0.28220*I 4.3141e-32 + 0.038862*I -4.9304e-32 - 0.52554*I     0.14607 + 0.24334*I]
[    0.13760 - 0.66033*I  2.2187e-31 - 0.91722*I -5.9165e-31 + 0.40344*I      1.1746 + 0.25689*I]


Before going any further, lat's define a function that will display the matrices in a rounded form so that we can compare them more easily.

In [13]:
def format_complex(z, digits=5, threshold=1e-10):
    real = float(z.real())
    imag = float(z.imag())
    
    # We round very small values to zero
    if abs(real) < threshold:
        real = 0
        if abs(imag) < threshold:
            return "0"
        # We format the result  
        return f"{imag:.{digits}f}*I"
    
    if abs(imag) < threshold:
        # We format the result
        return f"{real:.{digits}f}"

    sign = "+" if imag > 0 else "-"
    return f"{real:.{digits}f} {sign} {abs(imag):.{digits}f}*I"


def ApproxM(matrix, digits=5, threshold=1e-10):
    rows, cols = matrix.nrows(), matrix.ncols()
    
    for i in range(rows):
        formatted_row = [format_complex(matrix[i,j], digits, threshold) for j in range(cols)]
        print("\t".join(formatted_row))

In [14]:
ApproxM(pM)

-0.62448 + 0.28220*I	0.03886*I	-0.52554*I	0.14607 + 0.24334*I
0.13760 - 0.66033*I	-0.91722*I	0.40344*I	1.17459 + 0.25689*I


In [15]:
ApproxM(MofInt1)

0.13760 - 0.66033*I	-0.91722*I	0.40344*I	1.17459 + 0.25689*I
-0.62448 + 0.28220*I	0.03886*I	-0.52554*I	0.14607 + 0.24334*I


In what follows we use the function `matrix_of_integral_values()` instead of `period_matrix()` because it allows us to calculate periodic matrices of the second kind.

In [16]:
# Extract the omega-periods (first two columns)
omega = MofInt1[:, 0:2]

# Extract the omega'-periods (last two columns)
omegaP = MofInt1[:, 2:4]

ApproxM(omega)
print()
ApproxM(omegaP)

0.13760 - 0.66033*I	-0.91722*I
-0.62448 + 0.28220*I	0.03886*I

0.40344*I	1.17459 + 0.25689*I
-0.52554*I	0.14607 + 0.24334*I


Now we calculate the matrix
```{math}
\tau=\omega^{-1}\omega'
```
which belongs to the Siegel upper half-space. Hence it should satisfy two conditions:
- Symmetry:
```{math}
\tau^T = \tau
```
- Positive definiteness of the imaginary part
```{math}
Im(\tau)>0
```

In [17]:
tau= omega.inverse() * omegaP

# Displaying the result
print(tau.n(digits=5))

[-0.28894 + 0.70313*I -0.12636 - 0.46286*I]
[-0.12636 - 0.46286*I  -0.25854 + 1.6328*I]


In [18]:
# Test of the symmetry
print(tau-tau.transpose().n(digits=5))

[0.00000 0.00000]
[0.00000 0.00000]


In [19]:
# Test of positivity
# Calculating the complex part of the tau matrix
tauImag = tau.apply_map(lambda x: x.imag())

# Calculate the eigenvalues
eigenvalues = tauImag.eigenvalues()

# Checking if all eigenvalues are positive
all_positive = all(e > 0 for e in eigenvalues)

# Displaying the result
eigenvalues, all_positive

([1.8239307353738361091933820951, 0.51198440541283078719843599741], True)

### Second Kind Periods

To calculate the second kind period matrices, we need to calculate the following integrals along the canonical homology cycles $\{ \mathfrak{a}_i, \mathfrak{b}_i\}_{i=1}^g$
```{math}
    \eta = (\eta_{ij})= \left( \int_{\mathfrak{a}_j}du_i \right), \quad \eta' = (\eta'_{ij})= \left( \int_{\mathfrak{b}_j}du_i \right). 
```

In [20]:
# meromorphic differentials base
merbais=[x^2, 3*l0*x^3 + 2*l2*x^2 + l4*x]

In [21]:
MofInt2=S.matrix_of_integral_values(merbais)
# Let's display the matrix in a shortened form so that it will be easy to see its structure
ApproxM(MofInt2)

0.15574 + 0.20627*I	0.08372*I	-0.32882*I	-7.07532 + 0.12255*I
4.74867 + 3.09497*I	-0.04832*I	-6.23827*I	-2.96268 + 3.14329*I


In [22]:
# Extract the omega-periods (first two columns)
eta = MofInt2[:, 0:2]

# Extract the omega'-periods (last two columns)
etaP = MofInt2[:, 2:4]

ApproxM(eta)
print()
ApproxM(etaP)

0.15574 + 0.20627*I	0.08372*I
4.74867 + 3.09497*I	-0.04832*I

-0.32882*I	-7.07532 + 0.12255*I
-6.23827*I	-2.96268 + 3.14329*I


We can compute $\kappa$, given by
```{math}
    \kappa=\eta\; \omega^{-1}
```

In [23]:
omega_inv=Matrix(omega).inverse()
kappa = eta*omega_inv
ApproxM(kappa)

-0.09763 - 0.01261*I	-0.14978 - 0.29754*I
-0.14978 - 0.29754*I	-4.77835 - 7.02263*I


## Legendre relation

Now we can make another test. The not normalized period matrices of the first $\omega$, $\omega'$ and second $\eta$, $\eta'$ kinds should satisfy the Legendre relation
 ```{math}
\Omega^T J \Omega = 2\pi i J
```
    where
```{math}
\Omega=
    \begin{pmatrix}
        \omega && \omega'\\
        \eta && \eta'\\
    \end{pmatrix}, \quad 
J=        
    \begin{pmatrix}
        0 && -1_g'\\
        1_g && 0\\
    \end{pmatrix}
```

In [24]:
# Omega matrix
Omega = block_matrix([
    [omega, omegaP],
    [eta, etaP]
])

# Converting lists to matrices
zeroM = Matrix([[0.0, 0.0], [0.0, 0.0]])
mOneg = Matrix([[-1.0, 0.0], [0.0, -1.0]])
Oneg = Matrix([[1.0, 0.0], [0.0, 1.0]])

# J matrix
J = block_matrix([
    [zeroM, mOneg],
    [Oneg, zeroM]
])    


print("Omega Matrix:")
print(Omega.n(digits=5))
print()
print("J Matrix:")
print(J.n(digits=5))

Omega Matrix:
[     0.13760 - 0.66033*I  -4.9304e-32 - 0.91722*I| -3.9443e-31 + 0.40344*I       1.1746 + 0.25689*I]
[    -0.62448 + 0.28220*I -5.5467e-32 + 0.038862*I| -4.9304e-32 - 0.52554*I      0.14607 + 0.24334*I]
[-------------------------------------------------+-------------------------------------------------]
[     0.15574 + 0.20627*I  7.8886e-31 + 0.083724*I| -8.3816e-31 - 0.32882*I      -7.0753 + 0.12255*I]
[       4.7487 + 3.0950*I  1.1360e-28 - 0.048319*I|   2.2088e-29 - 6.2383*I       -2.9627 + 3.1433*I]

J Matrix:
[0.00000 0.00000|-1.0000 0.00000]
[0.00000 0.00000|0.00000 -1.0000]
[---------------+---------------]
[ 1.0000 0.00000|0.00000 0.00000]
[0.00000  1.0000|0.00000 0.00000]


In [25]:
import numpy as np

pi = np.pi
left=Omega.transpose()*J*Omega
right = 2*pi*I*J
result = left - right
ApproxM(result)

0	0	0	0
0	0	0	0
0	0	0	0
0	0	0	0


## Theta function

````{prf:definition}
:label: theta_df

A Riemann <i>theta function</i> $\theta(v;\tau)$ defined in terms of normalized coordinates $v$ and normalized period matrix $\tau$, canonicaly is given by
```{math}
\theta(\mathbf{v};\tau) = \sum_{n\in\mathbb{Z}^g} e^{i\pi \mathbf{n}^T \tau \mathbf{n} + 2i \pi \mathbf{n}^T \mathbf{v}}.
```
A theta function with characteristic $[\varepsilon ]$ is defined by
```{math}
\theta[\varepsilon](\mathbf{v};\tau) = 
    \theta 
    \begin{bmatrix}
            \varepsilon'_1 & \ldots & \varepsilon'_g\\
            \varepsilon_1 & \ldots & \varepsilon_g\\
    \end{bmatrix}
    (\mathbf{v};\tau)=
    \sum_{n\in\mathbb{Z}^g} e^{i\pi\{ (\mathbf{n}+ \frac{1}{2}\mathbf{\varepsilon}'^T ) \tau (\mathbf{n}+ \frac{1}{2}\mathbf{\varepsilon}') +2 (\mathbf{v}+\frac{1}{2}\mathbf{\varepsilon})^T (\mathbf{n}+\frac{1}{2}\mathbf{\varepsilon}') \} }.
```
where a characteristic all are half integer
```{math}
            \varepsilon_i, \varepsilon_k' = 1 \;\text{or} \; 0
```
````

````{important}
In the literature, it is common to use a convention with slightly different characteristics:
```{math}
(\varepsilon_i, \varepsilon'_j) \to 2 (\varepsilon_i, \varepsilon'_j)
```
hence  
```{math}
\varepsilon_i, \varepsilon'_j = \frac{1}{2} \;\text{or} \; 0.
```
        In consequence
```{math}
\theta[\varepsilon](\mathbf{v};\tau) = 
    \theta 
    \begin{bmatrix}
        \varepsilon'_1 & \ldots & \varepsilon'_g\\
        \varepsilon_1 & \ldots & \varepsilon_g\\
    \end{bmatrix}
    (\mathbf{v};\tau)=
    \sum_{n\in\mathbb{Z}^g} e^{i\pi\{ (\mathbf{n}+ \mathbf{\varepsilon}'^T ) \tau (\mathbf{n}+ \mathbf{\varepsilon}') +2 (\mathbf{v}+\mathbf{\varepsilon})^T (\mathbf{n}+\mathbf{\varepsilon}') \} }.
```
````

Let's code the $\theta$ function with the characteristic 

In [26]:
def ThetaCh(epsilon_m, v, ttau, NAcc):
    # NAcc is responsible for the number of elements in the sum, i.e. the precision of the result. 
    # Experimentally, a good approximation is obtained for NAcc>4, but of course this can be increased as needed.
    total_sum = 0
    # epsilon_m is the list [epsilon 1, epsilon 2] where epsilon1 and epsilon2 are vectors
    epsilon1 = epsilon_m[0]
    epsilon2 = epsilon_m[1]
    
    # We iterate over two indices from -NAcc to NAcc
    for n1 in range(-NAcc, NAcc):
        for n2 in range(-NAcc, NAcc):
            # We create vector n
            n = vector([n1, n2])
                    
            # The first component of the sum
            term1 = I * pi * (n + 1/2 * vector(epsilon1)) * (ttau * (n + 1/2 * vector(epsilon1)))
                    
            # The second component of the sum
            term2 = 2 * I * pi * (n + 1/2 * vector(epsilon1)) * (v + 1/2 * vector(epsilon2))
                    
            # We add the exp from these components to the total
            total_sum += exp(term1 + term2)
    
    return total_sum

````{note}
In Wolfram Mathematica exists a corresponding function under the name `SiegelTheta[$\nu_1$,$\nu_2$]($\Omega,s$)`. The relation between our variables and those in Mathematica are as follows
- $\Omega=\tau$
- $s=v$
- $\nu_1 = \frac{1}{2} epsilon1$ 
- $\nu_2 = \frac{1}{2} epsilon2$  
  
To test this, you can check the following code in Mathematica
```{code-block}
:caption: Mathematica Code      
            SiegelTheta[{{2, 4}, {4, 2}}, IdentityMatrix[2] I, {1, 2}] // N
```
and compare it with below one in SageMath
```{code-block}
:caption: SageMath Code              
            # Defining sample data
            epsilon_m = [vector([4, 8]), vector([8, 4])]
            v = vector([1, 2])
            tau = Matrix([[I, 0], [0, I]])
            NAcc = 5
            # Function call
            result = ThetaCh(epsilon_m, v, tau, NAcc)
            print(result)
```
  
Additionally, one can define the $\theta$ function in Mathematica in a similar way with the following code:
```{code-block}
:caption: Mathematica Code    
        ThetaCh[\[Epsilon]m_, v_, \[Tau]_, NAcc_] :=
         Sum[
          Exp[
           I Pi ({Subscript[n, 1], Subscript[n, 2]} + 
            1/2 \[Epsilon]m[[1]]) . (\[Tau] . ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]])) + 2 I Pi ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]]) . (v + 1/2 \[Epsilon]m[[2]])
           ],
          {Subscript[n, 2], -NAcc, NAcc}, {Subscript[n, 1], -NAcc, NAcc}
          ]
```
  
And check the result of the above test with: 
```{code-block}
:caption: Mathematica Code   
        ThetaCh[2 {{2, 4}, {4, 2}}, {1, 2}, IdentityMatrix[2] I, 5] // N
```
````

## Characteristics of branch points

In [27]:
# We define the genus variable
genus = S.genus  


eChars = [[[0 for k in range(genus)], [1 for k in range(genus)]], 
          [[0 for k in range(genus)] for i in range(2)]]

# Do-like loop in Julia's Mathematica notebook
for l in range(genus):
    # let's note the indexing, which must be adjusted by
    eChars.insert(0, 
        [[(eChars[0][0][k] + kronecker_delta(k+1, genus - l) + 
           kronecker_delta(k+1, genus - l + 1)) % 2 for k in range(genus)],
         [(eChars[0][1][k] + 0) % 2 for k in range(genus)]]
    )

    eChars.insert(0,
        [[(eChars[0][0][k] + 0) % 2 for k in range(genus)],
         [(eChars[0][1][k] + kronecker_delta(k+1, genus - l)) % 2 for k in range(genus)]]
    )

# We display matrices
seen_matrices = []
for i in range(len(eChars)):
    current_matrix = matrix(eChars[i])
    if current_matrix not in seen_matrices:
        seen_matrices.append(current_matrix)
        print(current_matrix)
        print()

[1 0]
[0 0]

[1 0]
[1 0]

[0 1]
[1 0]

[0 1]
[1 1]

[0 0]
[1 1]

[0 0]
[0 0]



```{note}
This result can be compared with the result at work {cite}`enolski_periods_2007` (p. 14, example 5.1)
```  

In [28]:
# We sum the eChars elements with indices 2*i +1 (because python counts from 0) and take Mod 2
KCh = sum(matrix(eChars[2 * i+1]) for i in range(genus)) % 2

print(KCh)

[1 1]
[0 1]


## $\sigma$-Functions

````{prf:definition}
:label: sigma_df

<em> Sigma function </em> (Kleinian sigma) is a modular invariant entire function on $\mathrm{Jac}(\mathscr{C})$. It is definef by a relation with the theta function:
```{math}
    \sigma(\mathbf{u}) = C \tilde{\sigma}(\mathbf{u})
```
        where
```{math}
    \tilde{\sigma}(\mathbf{u})= e^{-\frac{1}{2}\mathbf{u}^T \kappa \mathbf{u}} \theta[ K ] (\omega^{-1} \mathbf{u}, \omega^{-1} \omega'),
```
```{math}
    C= \sqrt{\frac{\pi^g}{\det{\omega}}} \left( \prod_{1\leq i<j \leq 2g+1} (e_i - e_j) \right)^{-1/4},
```
        and $[K]$ denotes the characteristics of the vector of Riemann constants. (The expression for $C$ comes from {cite}`enolski_inversion_2012`, p.9, Eq. II.41)
````

```{note}
    The assumed formula for the constant $C$ is burdened with uncertainty because in another classical source {cite}`buchstaber_hyperelliptic_1996` one can find a different formulation for it. However, it will not affect the final formulas in the following parts of the code because it will be contracted.    
```  

In [29]:
# We define variables
var('U1 U3')

# We define the accuracy of theta function
Acc=20


# sigma
def Tsigma(U1, U3):
    e = exp(-(1/2)*(vector([U1, U3])*kappa*vector([U1, U3])))
    theta = ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)
    return e*theta

# C constant
det_omega = omega.determinant()
g = 2
#Branch points
BP=find_branch_points(l0, l2, l4, l6, l8, l10)
e1, e2, e3, e4, e5 = BP[0], BP[1], BP[2], BP[3], BP[4]
prod=(e1-e2)*(e2-e3)*(e3-e4)*(e4-e5)
C = sqrt(pi**g / det_omega) * prod**(-1/4)

def sigma(U1, U3):
    return C*Tsigma(U1, U3)

###  Approximate definitions

````{prf:definition}
:label: sigma_app1_df

Based on the work {cite}`donagi_-functions_2020` p.20, Eq. (3.22), we can write first approximation of $\mathbf{u}=(u_1,u_3)$ for small values
```{math}
        \sigma(\mathbf{u}) = u_3 - \frac{1}{3} u_1^2 + \frac{1}{6}\lambda_6 u_3^3 - \frac{1}{12}\lambda_4 u_1^4 u_3 - \frac{1}{6} \lambda_6 u_1^3 u_3^2 - \frac{1}{6} \lambda_8 u_1^2 u_3^3 - \frac{1}{3} \lambda_{10} u_1 u_3^4 +\left( \frac{1}{60} \lambda_4 \lambda_8 + \frac{1}{120} \lambda_6^2\right) u_3^5 +(u^7).
```
````    

In [30]:
def sigmaApp1(U1, U3):
    result = (U3 
              - (1/3) * U1**3 
              + (1/6) * l6 * U3**3 
              - (1/12) * l4 * U1**4 * U3 
              - (1/6) * l6 * U1**3 * U3**2
              - (1/6) * l8 * U1**2 * U3**3 
              - (1/3) * l10 * U1 * U3**4 
              + (1/60) * l4 * l8 * U3**5 
              + (1/120) * l6**2 * U3**5)
    return result

````{prf:definition}
:label: sigma_app2_df

Based on the work {cite}`eilbeck_theory_2017` p.34, Eq. (3.22), we can write first approximation of $\mathbf{u}=(u_1,u_3)$ for small values
```{math}
    \begin{aligned}
        \sigma(\mathbf{u}) = & u_3 - \frac{1}{3} u_1^2 + \frac{1}{6}\lambda_6 u_3^3 - \frac{1}{12}\lambda_4 u_1^4 u_3 - \frac{1}{6} \lambda_6 u_1^3 u_3^2 - \frac{1}{6} \lambda_8 u_1^2 u_3^3 - \frac{1}{3} \lambda_{10} u_1 u_3^4 +\left( \frac{1}{60} \lambda_4 \lambda_8 + \frac{1}{120} \lambda_6^2\right) u_3^5 +(u^7) \\
        & - \frac{4}{7!} \lambda_4 *u_1^7 + \frac{69}{9!} \lambda_6 u_1^9 - \frac{8}{6!}\lambda_6 u_3 u_1^6 + (1600\lambda_8 - 408 \lambda_4^2) \frac{u^11}{11!} - (4\lambda_4^2 + 32 \lambda_8)\frac{u_3 u_1^8}{8!} - \frac{8}{2! 5!} \lambda_8 u_3^2 u_1^5.  
    \end{aligned}    
```
````      

In [31]:
def sigmaApp2(U1, U3):
    result = (U3 
              - (1/3) * U1**3 
              + (1/6) * l6 * U3**3 
              - (1/12) * l4 * U1**4 * U3 
              - (1/6) * l6 * U1**3 * U3**2
              - (1/6) * l8 * U1**2 * U3**3 
              - (1/3) * l10 * U1 * U3**4 
              + (1/60) * l4 * l8 * U3**5 
              + (1/120) * l6**2 * U3**5
              - (4/factorial(7)) * l4 * U1**7
              + (64/factorial(9)) * l6 * U1**9
              - (8/factorial(6)) * l6 * U3 * U1**6
              + (1600 * l8 - 408 * l4**2) * (1/factorial(11)) * U1**11
              - (4 * l4**2 + 32 * l8) * (1/factorial(8)) * U3 * U1**8
              - (4/factorial(5)) * l8 * U3**2 * U1**5
             )
    
    return result

### Tests

Let's check how approximate sigmas relate to the exact definition. We operate on the ratios to get rid of the dependence on the constant $C$

In [32]:
sigmaApp1(0.00015,0.00023)/sigmaApp1(0.000071,0.000091)

2.52747253533945

In [33]:
sigma(0.00015,0.00023)/sigma(0.000071,0.000091)

2.27708680068096 - 6.52811138479592e-14*I

In [34]:
u1=2.21231
u3=7.32112

k1=8.8976
k3=5.2315

In [35]:
d=10^(-2)

testS = sigma(u1*d,u3*d)/sigma(k1*d,k3*d)
testSApp1 = sigmaApp1(u1*d,u3*d)/sigmaApp1(k1*d,k3*d)
testSApp2 = sigmaApp2(u1*d,u3*d)/sigmaApp2(k1*d,k3*d)

print("Result for comparison with sigmaApp1:")
print(testS-testSApp1)
print()
print("Result for comparison with sigmaApp2:")
print(testS-testSApp2)
print()
print("Difference between sigmaApp1 and sigmaApp2:")
print(testSApp1-testSApp2)

Result for comparison with sigmaApp1:
-0.918639409348416 - 1.11022302462516e-16*I

Result for comparison with sigmaApp2:
-0.918639706374918 - 1.11022302462516e-16*I

Difference between sigmaApp1 and sigmaApp2:
-2.97026502282804e-7


In [36]:
d=10^(-5)

testS = sigma(u1*d,u3*d)/sigma(k1*d,k3*d)
testSApp1 = sigmaApp1(u1*d,u3*d)/sigmaApp1(k1*d,k3*d)
testSApp2 = sigmaApp2(u1*d,u3*d)/sigmaApp2(k1*d,k3*d)

print("Result for comparison with sigmaApp1:")
print(testS-testSApp1)
print()
print("Result for comparison with sigmaApp2:")
print(testS-testSApp2)
print()
print("Difference between sigmaApp1 and sigmaApp2:")
print(testSApp1-testSApp2)

Result for comparison with sigmaApp1:
-0.884388199106624 + 8.44047054471275e-14*I

Result for comparison with sigmaApp2:
-0.884388199106624 + 8.44047054471275e-14*I

Difference between sigmaApp1 and sigmaApp2:
0.000000000000000


In [37]:
d=10^(-10)

testS = sigma(u1*d,u3*d)/sigma(k1*d,k3*d)
testSApp1 = sigmaApp1(u1*d,u3*d)/sigmaApp1(k1*d,k3*d)
testSApp2 = sigmaApp2(u1*d,u3*d)/sigmaApp2(k1*d,k3*d)

print("Result for comparison with sigmaApp1:")
print(testS-testSApp1)
print()
print("Result for comparison with sigmaApp2:")
print(testS-testSApp2)
print()
print("Difference between sigmaApp1 and sigmaApp2:")
print(testSApp1-testSApp2)

Result for comparison with sigmaApp1:


-0.884388165458948 - 1.88429921804678e-8*I

Result for comparison with sigmaApp2:
-0.884388165458948 - 1.88429921804678e-8*I

Difference between sigmaApp1 and sigmaApp2:
0.000000000000000


In [38]:
d=10^(-15)

testS = sigma(u1*d,u3*d)/sigma(k1*d,k3*d)
testSApp1 = sigmaApp1(u1*d,u3*d)/sigmaApp1(k1*d,k3*d)
testSApp2 = sigmaApp2(u1*d,u3*d)/sigmaApp2(k1*d,k3*d)

print("Result for comparison with sigmaApp1:")
print(testS-testSApp1)
print()
print("Result for comparison with sigmaApp2:")
print(testS-testSApp2)
print()
print("Difference between sigmaApp1 and sigmaApp2:")
print(testSApp1-testSApp2)

Result for comparison with sigmaApp1:
-0.885876764827322 - 0.00180870273726220*I

Result for comparison with sigmaApp2:
-0.885876764827322 - 0.00180870273726220*I

Difference between sigmaApp1 and sigmaApp2:
0.000000000000000


It seems that both approximations are similar. 
  
Let's check the $C$ constant. Since in the first order the function $\sigma$ is linear in $u_1$:
```{math}
        \sigma(\mathbf{u}) = u_3 + \ldots
```
then for $u_1\approx0$ we can estimate:
```{math}
        C \approx \frac{u_3}{\tilde{\sigma}(\mathbf{u})}.
```

In [39]:
u1=10^(-20)
u3=0.0012131

Capp=u3/Tsigma(u1,u3)
print("C from definition:")
print(C)
print()
print("Approximate C:")
print(Capp)

C from definition:
1.37901860572385 + 1.32549303772618*I

Approximate C:
-0.394449548035149 + 0.592130183781794*I


The result doesn't match, so it's something that can be improved in the future

## $\wp$-Functions

````{prf:definition}
:label: wp_df

Multiply periodic Klein-Weierstrass $\wp$-functions are defined by
```{math}
    \wp_{ij}(\mathbf{u}):= - \frac{\partial^2\log{\sigma(\mathbf{u})}}{\partial u_i \partial u_j}, \quad \wp_{ijk}:=- \frac{\partial^3\log{\sigma(\mathbf{u})}}{\partial u_i \partial u_j \partial u_k},
```
where $\sigma(u)$ is called <i>sigma function</i>. This can be also written by 
```{math}
    \wp_{ij} = \frac{\sigma_i(\mathbf{u})\sigma_j(\mathbf{u}) - \sigma(\mathbf{u})\sigma_{ij}(\mathbf{u})}{\sigma^2(\mathbf{u})} = \frac{\tilde{\sigma}_i(\mathbf{u})\tilde{\sigma}_j(\mathbf{u}) - \tilde{\sigma}(\mathbf{u})\tilde{\sigma}_{ij}(\mathbf{u})}{\tilde{\sigma}^2(\mathbf{u})},
```
where $\sigma_i(\mathbf{u})$ denotes the derivative of the sigma function with respect to the $i$-th component of $\mathbf{u}$. 
  
It can be shown that the above definitions can be written in the form
```{math}
    \wp_{ij}:=\kappa_{ij} - \frac{\partial^2}{\partial u_i \partial u_j}\log{\theta[K](\omega^{-1}u;\tau)}, \quad \wp_{ijk}:=- \frac{\partial^3}{\partial u_i \partial u_j \partial u_k}\log{\theta[K](\omega^{-1}u;\tau)}.
```
````        

In [40]:
#definition with thetas
# We define variables
var('U1 U3')

# We define the accuracy of theta function
Acc=20


# WeierstrassP11
def WeierstrassP11(u1_val, u3_val):
    symbolic_expr = kappa[0, 0] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, 2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP13
def WeierstrassP13(u1_val, u3_val):
    symbolic_expr = kappa[0, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)),  U1, U3)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP33
def WeierstrassP33(u1_val, u3_val):
    symbolic_expr = kappa[1, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U3, 2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP3333
def WeierstrassP3333(u1_val, u3_val):
    symbolic_expr = kappa[1, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U3, 4)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()


In [41]:
# Definition with sigmas


# WeierstrassP11
def Weierstrass2P11(u1_val, u3_val):
    U1, U3 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma1 = diff(sigma_expr, U1)
    sigma11 = diff(sigma_expr, U1, 2)
    symbolic_expr = (sigma1*sigma1 - sigma_expr*sigma11)/(sigma_expr^2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP13
def Weierstrass2P13(u1_val, u3_val):
    U1, U3 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma1 = diff(sigma_expr, U1)
    sigma3 = diff(sigma_expr, U3)
    sigma13 = diff(sigma_expr, U1, U3)
    symbolic_expr = (sigma1*sigma3 - sigma_expr*sigma13)/(sigma_expr^2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()
    
# WeierstrassP33
def Weierstrass2P33(u1_val, u3_val):
    U1, U3 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma3 = diff(sigma_expr, U3)
    sigma33 = diff(sigma_expr, U3, 2)
    symbolic_expr = (sigma3*sigma3 - sigma_expr*sigma33)/(sigma_expr^2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()    

### Tests

As a first test of these functions, we can check if they satisfy the key property  
```{math}
    \wp_{ij}(\mathbf{u}+2\omega \mathbf{n} + 2\omega'\mathbf{n}') = \wp_{ij}(\mathbf{u}),
```
where
```{math}
    \mathbf{n} = 
        \begin{pmatrix}
            n_1\\
            n_2
        \end{pmatrix}, \quad
    \mathbf{n}' = 
        \begin{pmatrix}
            n'_1\\
            n'_2
        \end{pmatrix} \in \mathbb{Z}^2.
```

In [42]:
ntest = vector([1, 2])
nPtest = vector([-3, -5])

wn = omega*ntest
wPn= omegaP*nPtest

print("Theta based definitions")
print()
print("Test P11:")
print(WeierstrassP11(2.0, 3.0) - WeierstrassP11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(WeierstrassP13(2.0, 3.0) - WeierstrassP13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(WeierstrassP33(2.0, 3.0) - WeierstrassP33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print()
print("Sigma based definitions")
print()
print("Test P11:")
print(Weierstrass2P11(2.0, 3.0) - Weierstrass2P11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(Weierstrass2P13(2.0, 3.0) - Weierstrass2P13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(Weierstrass2P33(2.0, 3.0) - Weierstrass2P33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))

Theta based definitions

Test P11:


-2.20040874410188e-10 - 3.79429820895893e-12*I

Test P13:


-3.93356458516791e-11 - 8.13997758086771e-11*I

Test P33:


3.04964942188235e-11 - 4.54747350886464e-11*I


Sigma based definitions

Test P11:


-5.12834219534852e-12 + 1.49480428035531e-12*I

Test P13:


1.05293551655450e-12 - 9.41107986716367e-13*I

Test P33:


-5.14432940690313e-12 - 7.08766378920700e-13*I


In [43]:
ntest = vector([10, -30])
nPtest = vector([3, -3])

wn = omega*ntest
wPn= omegaP*nPtest

print("Theta based definitions")
print()
print("Test P11:")
print(WeierstrassP11(2.0, 3.0) - WeierstrassP11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(WeierstrassP13(2.0, 3.0) - WeierstrassP13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(WeierstrassP33(2.0, 3.0) - WeierstrassP33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print()
print("Sigma based definitions")
print()
print("Test P11:")
print(Weierstrass2P11(2.0, 3.0) - Weierstrass2P11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(Weierstrass2P13(2.0, 3.0) - Weierstrass2P13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(Weierstrass2P33(2.0, 3.0) - Weierstrass2P33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))

Theta based definitions

Test P11:


1.53477230924182e-12 + 3.56692453351570e-12*I

Test P13:


2.06341610464733e-11 - 4.76347850053571e-11*I

Test P33:


-2.78532752417959e-12 + 1.07434061646927e-11*I


Sigma based definitions

Test P11:


NaN + 5.66657831768680e-13*I

Test P13:


NaN - 5.54251612982271e-15*I

Test P33:


NaN - +infinity*I


```{important}
I'm not sure why (probably from the way SageMath deals with numerics) the above test is satisfied for arbitrary vectors $\mathbf{n}$ however the coordinates of vectors $\mathbf{n}'$ should be "small", i.e. $|n'_i|<4$.     
```

In [44]:
ntest = vector([1,2])
nPtest = vector([4, -4])

wn = omega*ntest
wPn= omegaP*nPtest

print("Theta based definitions")
print()
print("Test P11:")
print(WeierstrassP11(2.0, 3.0) - WeierstrassP11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(WeierstrassP13(2.0, 3.0) - WeierstrassP13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(WeierstrassP33(2.0, 3.0) - WeierstrassP33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print()
print("Sigma based definitions")
print()
print("Test P11:")
print(Weierstrass2P11(2.0, 3.0) - Weierstrass2P11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(Weierstrass2P13(2.0, 3.0) - Weierstrass2P13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(Weierstrass2P33(2.0, 3.0) - Weierstrass2P33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))

Theta based definitions

Test P11:


1455.83379737340 - 304.287205828748*I

Test P13:


1590.65313919950 - 3793.73914184472*I

Test P33:


-6146.83000392507 - 9574.88959406405*I


Sigma based definitions

Test P11:


NaN + 5.66657831768680e-13*I

Test P13:


NaN - 5.54251612982271e-15*I

Test P33:


NaN - 1.59872115546023e-13*I


````{prf:observation}
So neither of them is perfect, but for now the definition based directly on the theta function seems to work better.
````

```{note}
For future tests it would be good to derive one of the classical relations between derivatives of the function $\wp_{ij}$ for our specific parameterisation of the curve. Something similar like in {cite}`athorne_sl2_2004`.
    </p>
```

## Jacobi inversion problem on branch points

Let $\mathbf{u}=\mathcal{A}(D)$ be the Abel image of a degree g positive non-special divisor $D$ on the curve. Then $D$ is uniquely defined by the system of equations:
```{math}
        \mathcal{R}_{2g}(x;\mathbf{u}) = x^g - \sum_{i=1}^g x^{g-1} \wp_{1,2i-1}(\mathbf{u}) =0,
```
```{math}
        \mathcal{R}_{2g+1}(x,y;\mathbf{u}) = 2y + \sum_{i=1}^{g-i} x^{g-1} \wp_{1,1,2i-1}(\mathbf{u}) =0.
```
So in case of $g=2$:    
```{math}
        \mathcal{R}_4(x;\mathbf{u})=x^2 - x\wp_{11}(\mathbf{u})- \wp_{13}(\mathbf{u})=0,
```   
```{math}
        \mathcal{R}_5 (x;\mathbf{u})= 2y + x\wp_{111}(\mathbf{u}) + \wp_{113}(\mathbf{u})=0.
```

````{prf:definition}
:label: divisor_df

Let 
```{math}
        D=\sum_{i=1}^n P_i,
```
be a divisor on a curve $\mathscr{C}$. We assume that $D$ is <b>non-special</b>, that is 
- $ n\geq g$,
- $D$ does not contain pairs of points connected by the hyperelliptic involution.

````

```{note}

On special divisors $\sigma$-function vanishes, and so $\wp$-functions have singularities.

```

As a point $P_i$ we choose
```{math}
    P_i = (x_i,y_i) = (x_i, y_{+}(x_i)),
```
where $y_{+}(x) = + \sqrt{y^2(x)}$ (there is second option $y_{-}(x) = - \sqrt{y^2(x)}$ ). 

In our case let's take
```{math}
    P_1 = (x_1, y_1)=(1.5,y_{+}(1.5))
```
```{math}
    P_2 = (x_2, y_2)=(3.1,y_{+}(3.1))
```

In [45]:
# Definition of y function 
def y(x):
    res = sqrt(l0*x^5 + l2*x^4 + l4*x^3 + l6*x^2 + l8*x + l10)
    return res.n()

To calculate $\mathbf{u}$ I will use the Sage function:  `abel_jacobi()`, where it expects lists of tuples in the format  `(v,P)`, where  `v` is the multiplicity of the point in the divisor (in our case 1 for both points), and  `P` is a tuple  `(x,y)` representing the point on the curve. 
  
The multiplicity `v` (also called valuation) in the context of divisors determines how many times a given point appears in the divisor. Here are some key points:
- For regular points on the curve that are neither singular points nor points at infinity, typically $v = 1$ or $v = -1$ is used.
$v = 1$ means the point appears positively in the divisor (it is "added").
- $v = -1$ means the point appears negatively in the divisor (it is "subtracted").
- If a point appears multiple times, $v$ can be greater than 1 or less than -1.
- For special points (e.g., points at infinity or singular points), v may take other values depending on the local structure of the curve at that point.
  
  
In your case, where you define the divisor as a sum of two regular points: `D = [x1 + x2, y1 + y2]` each of these points appears once positively in the divisor, so for both points`v = 1`. Therefore, divisor in the format suitable for the `abel_jacobi()` function would look like this:  

In [46]:
x1 = 0.256
x2 = 11.721

y1 = y(x1)
y2 = y(x2)

# P_i points
P1 = [ x1, y1 ] 
P2 = [ x2, y2 ]

# Divisor
divisor = [(1, (x1, y1)), (1, (x2, y2))]

```{note}
If one wanted to define a more complex divisor, for example, the difference of two points, it might look like this:  
`divisor = [(1, (x1, y1)), (-1, (x2, y2))]`  
where -1 indicates that the second point is subtracted in the divisor.
```

In [47]:
AJ = S.abel_jacobi(divisor)

print(AJ)

(-0.095227328487916328568064649224 - 0.47103938123276306779408678278*I, -0.94610300133296732955826058561 - 0.36651365565790567383047992719*I)


### Tests

In [48]:
u1=AJ[0]
u3=AJ[1]

print("P11(u):")
print(WeierstrassP11(u1, u3))

print("P13(u):")
print(WeierstrassP13(u1, u3))

print("P33(u):")
print(WeierstrassP11(u1, u3))

P11(u):


6.36829653384861 - 4.59081552648913*I
P13(u):


-9.85938695314932 - 1.97507933943491*I
P33(u):


6.36829653384861 - 4.59081552648913*I


In [49]:
mat1R4=[[1, x, x^2],[ 1, x1, x1^2], [1, x2, x2^2]]
det1=matrix(mat1R4).determinant()
mat2R4=[[ 1, x1], [1, x2]]
det2=matrix(mat2R4).determinant()
det1/det2

x^2 - 11.9770000000000*x + 3.00057600000000

````{prf:observation}
Ok, this is bad. Not only are the results bad, but $\wp_{11}$ and $\wp_{33}$ are identical, and that shouldn't be the case.
````

## Bibliography

```{bibliography}
:style: unsrt
:filter: docname in docnames
```